In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report,accuracy_score

In [3]:
dataset=pd.read_csv('Final_dataset1.csv')

In [4]:
dataset.head(5)

,Flow Duration,Flow Bytes/s,Flow Packets/s,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Bwd Packet Length Mean,Max Packet Length,...,Subflow Bwd Packets,Subflow Fwd Bytes,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Mean,Active Std,Idle Mean,Idle Std,Label
0,1.0,2.944000e+09,2000000.0,2.0,0.0,2944.0,0.0,1472.0,0.0,1472.0,...,0.0,2944.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,DrDoS_DNS
1,1.0,2.944000e+09,2000000.0,2.0,0.0,2944.0,0.0,1472.0,0.0,1472.0,...,0.0,2944.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,DrDoS_DNS
2,1.0,2.944000e+09,2000000.0,2.0,0.0,2944.0,0.0,1472.0,0.0,1472.0,...,0.0,2944.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,DrDoS_DNS
3,1.0,2.944000e+09,2000000.0,2.0,0.0,2944.0,0.0,1472.0,0.0,1472.0,...,0.0,2944.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,DrDoS_DNS
4,1.0,2.944000e+09,2000000.0,2.0,0.0,2944.0,0.0,1472.0,0.0,1472.0,...,0.0,2944.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,DrDoS_DNS


In [5]:
dataset.shape

(1600000, 33)

In [8]:
X = dataset.drop("Label", axis=1)
y = dataset["Label"]

In [10]:
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
base_model = DecisionTreeClassifier()

In [17]:
bagging_model = BaggingClassifier(
    estimator=base_model,
    n_estimators=20,
    max_samples=0.7,
    bootstrap=True,
    n_jobs=-1,                 # Use all CPU cores
    random_state=42
)

In [18]:
bagging_model.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.7,
                  n_estimators=20, n_jobs=-1, random_state=42)

In [19]:
y_pred = bagging_model.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

       BENIGN       1.00      1.00      1.00     40579
    DrDoS_DNS       0.78      0.30      0.43     39972
   DrDoS_LDAP       0.50      0.68      0.58     39969
  DrDoS_MSSQL       0.94      0.96      0.95     39985
    DrDoS_NTP       0.99      1.00      1.00     39531
DrDoS_NetBIOS       0.95      0.99      0.97     39986
   DrDoS_SNMP       0.68      0.84      0.75     39990
   DrDoS_SSDP       0.99      0.96      0.98     39988

     accuracy                           0.84    320000
    macro avg       0.85      0.84      0.83    320000
 weighted avg       0.85      0.84      0.83    320000



In [23]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8407


CatBoost

In [6]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [9]:
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    cat_features=categorical_features,
    verbose=100,
    random_seed=42
)


In [12]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

0:	learn: 0.7860633	test: 0.7848375	best: 0.7848375 (0)	total: 1.85s	remaining: 30m 49s
100:	learn: 0.8311352	test: 0.8309375	best: 0.8309375 (100)	total: 2m 50s	remaining: 25m 21s
200:	learn: 0.8341477	test: 0.8338156	best: 0.8338156 (200)	total: 5m 43s	remaining: 22m 43s
300:	learn: 0.8357867	test: 0.8354438	best: 0.8354438 (300)	total: 8m 40s	remaining: 20m 9s
400:	learn: 0.8364695	test: 0.8359937	best: 0.8359937 (400)	total: 11m 38s	remaining: 17m 23s
500:	learn: 0.8367461	test: 0.8362656	best: 0.8362688 (481)	total: 14m 27s	remaining: 14m 23s
600:	learn: 0.8369539	test: 0.8364563	best: 0.8364625 (595)	total: 17m 12s	remaining: 11m 25s
700:	learn: 0.8370586	test: 0.8365187	best: 0.8365281 (693)	total: 20m 2s	remaining: 8m 33s
800:	learn: 0.8371609	test: 0.8366219	best: 0.8366219 (800)	total: 22m 51s	remaining: 5m 40s
900:	learn: 0.8372266	test: 0.8366781	best: 0.8366938 (871)	total: 25m 42s	remaining: 2m 49s
999:	learn: 0.8372883	test: 0.8367906	best: 0.8367906 (999)	total: 28m 27s

In [13]:
y_pred = model.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {acc:.4f}")


Accuracy: 0.8368
